# Task-5
## Camera Calibration

In [45]:
#import required libraries
import numpy as np
import cv2
import cv2.aruco as aruco

#when specified accuracy or total number iterations is reached, stop.
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 28, 0.001)

#images used for calibration
images_names = images_names = ['checkerboard_1.jpg', 'checkerboard_2.jpg',  'checkerboard_3.jpg',  'checkerboard_4.jpg', 
                               'checkerboard_5.jpg', 'checkerboard_6.jpg',  'checkerboard_7.jpg',  'checkerboard_8.jpg', 
                               'checkerboard_9.jpg', 'checkerboard_10.jpg', 'checkerboard_11.jpg', 'checkerboard_12.jpg',
                              'checkerboard_13.jpg', 'checkerboard_14.jpg', 'checkerboard_15.jpg', 'checkerboard_16.jpg',
                              'checkerboard_17.jpg', 'checkerboard_18.jpg', 'checkerboard_19.jpg', 'checkerboard_20.jpg',
                              'checkerboard_21.jpg', 'checkerboard_22.jpg', 'checkerboard_23.jpg', 'checkerboard_24.jpg',
                              'checkerboard_25.jpg', 'checkerboard_26.jpg', 'checkerboard_27.jpg', 'checkerboard_28.jpg']

#checkerboard sixe (NOTE: 6, 9 are the "inside" corners)
Checkerboard = (6, 9)

#function to calibrate
#images_names -> list of image names stores
#square_size -> dimension of one square on the checkerboard
#width and height of checkerboard
def calibrate(images_names, square_size, width = Checkerboard[1], height = Checkerboard[0]):
    
    #checkerboard matrix
    objp = np.zeros((height*width, 3), np.float32)
    objp[:, :2] = np.mgrid[0:width, 0:height].T.reshape(-1, 2)
    
    #coordinates * square_size to get real world coordinates
    objp = objp * square_size
    
    #3d point in real world space
    objpoints = []
    
    # 2d points in image plane.
    imgpoints = []
    
    count = 0
    for name in images_names:
        count += 1
        img = cv2.imread(name)
        grayim = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Find the chess board corners
        ret, corners = cv2.findChessboardCorners(grayim, (width, height), None)
        
        #if found add it to objpoints and imgpoints
        if ret :
            objpoints.append(objp)

            corners2 = cv2.cornerSubPix(grayim, corners, (11, 11), (-1, -1), criteria)
            imgpoints.append(corners2)

            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, Checkerboard, corners2, ret)
            
            
            #show image with corners 
            cv2.imshow("image",img)
            k = cv2.waitKey(0)
            if (k == ord('s')):
                cv2.imwrite(f'checkerboard_output{count}.jpg',img)
                cv2.destroyAllWindows()
            elif (k == ord('q')):
                cv2.destroyAllWindows()            
    
    ret, camera_matrix, dist_coeff, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, grayim.shape[::-1], None, None)

    return [ret, camera_matrix, dist_coeff, rvecs, tvecs]

#square size = 15mm
ret, camera_matrix, dist_coeff, rvecs, tvecs = calibrate(images_names, 0.015)

print(f'Distortion coefficient matrix = \n{dist_coeff}')
print(f'Camera matrix = \n{camera_matrix}')


Distortion coefficient matrix = 
[[-0.2364114   1.02806359  0.00328902 -0.01250208 -1.7818219 ]]
Camera matrix = 
[[1.01412033e+03 0.00000000e+00 2.50986073e+02]
 [0.00000000e+00 1.00874818e+03 3.92568354e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


## Pose Estimation

In [43]:
import math

#start webcam video
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('result_task3.avi', fourcc, 20.0, (1280, 720))

while (True):
    ret, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    aruco_dict = aruco.Dictionary_get(aruco.DICT_7X7_1000)

    parameters = aruco.DetectorParameters_create()
    
    corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=parameters, cameraMatrix = camera_matrix, distCoeff = dist_coeff)
    
    font = cv2.FONT_HERSHEY_SIMPLEX

    # check if the ids list is not empty
    if (ids != None):
        # 0.1 = marker size
        ret = aruco.estimatePoseSingleMarkers(corners, 0.1, mtx, dist)
        
        #rvec = 1*3 matrix, tvec = 1*3 matrix
        rvec = np.matrix(ret[0][0,0,:]) 
        tvec = np.matrix(ret[1][0,0,:])
        tvec_transpose = tvec.T
        
        #use rodrigues to transform from camera rotation vector to camera rotation matrix
        rvec_matrix = cv2.Rodrigues(rvec)[0]
        #proj matrix = [R|t]
        proj_matrix = np.concatenate((rvec_matrix, tvec.T), axis = 1)
        #get euler angles to calculate pitch, yaw and roll
        euler_angles = cv2.decomposeProjectionMatrix(proj_matrix)[6]

        pitch, yaw, roll = [math.radians(angle) for angle in euler_angles]

        pitch = math.degrees(pitch)
        roll = math.degrees(roll)
        yaw = math.degrees(yaw)       
                
        aruco.drawDetectedMarkers(frame, corners)
                                                        
        aruco.drawAxis(frame, mtx, dist, rvec, tvec, 0.1)
    
        cv2.putText(frame, f"Marker position x = {tvec[0,0]:.2f}, y = {tvec[0,1]:.2f}, z = {tvec[0,2]:.2f}", (0, 100), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame, f"Marker pitch = {pitch:.2f}, roll = {roll:.2f}, yaw = {yaw:.2f} ", (0, 150), font, 1, (0, 255, 0), 2, cv2.LINE_AA)

    else:
        # code to show 'No Ids' when no markers are found
        cv2.putText(frame, "No Ids", (0,64), font, 1, (0,255,0),2,cv2.LINE_AA)
        
    out.write(frame)
    
    # display the resulting frame
    cv2.imshow('frame', frame)

    #use 'q' to quit
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

    
# When everything done, release the capture
cap.release()
out.release()
cv2.destroyAllWindows()
